In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np
import pandas as pd
from keras.utils import to_categorical

backend = tf.keras.backend
layers = tf.keras.layers

# Make identity Block

In [3]:
class Identity_Block(tf.keras.Model):
    def __init__(self, filters, dilation = 1):
        super(Identity_Block, self).__init__(name='')
    
        filters1, filters2, filters3 = filters
        
        
        self.conv1 = tf.keras.layers.Conv2D(filters1, kernel_size = (1, 1), strides = (1, 1)
                                            , padding='same')
        self.conv2 = tf.keras.layers.Conv2D(filters2, kernel_size = (3, 3), strides = (1, 1)
                                            , dilation_rate = 1, padding='same')
        self.conv3 = tf.keras.layers.Conv2D(filters3, kernel_size = (1, 1), strides = (1, 1)
                                            , padding='same')
        
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.bn3 = tf.keras.layers.BatchNormalization()
        
        self.relu = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
        
    def call(self, inputs, training = False):
        x = self.conv1(inputs)
        x = self.bn1(x, training = training)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        x = self.add([x, inputs])
        x = self.relu(x)
        
        return x

# Make Convolution Block

In [4]:
class Conv_Block(tf.keras.Model):
    def __init__(self, filters, strides = (2, 2), dilation = 1):
        super(Conv_Block, self).__init__()
        
        filters1, filters2, filters3 = filters
            
        self.conv1 = tf.keras.layers.Conv2D(filters1, kernel_size = (1, 1), strides = strides
                                            , padding='same')
        self.conv2 = tf.keras.layers.Conv2D(filters2, kernel_size = (3, 3), strides = (1, 1)
                                            , dilation_rate = dilation, padding='same')
        self.conv3 = tf.keras.layers.Conv2D(filters3, kernel_size = (1, 1), strides = (1, 1)
                                            , padding='same')  
        self.short_cut = tf.keras.layers.Conv2D(filters3, kernel_size = (1, 1)
                                               , strides = strides, padding = 'same')
        
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.bn4 = tf.keras.layers.BatchNormalization()
        
        self.relu = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        
        short_cut = self.short_cut(inputs)
        short_cut = self.bn4(short_cut)
        
        x = self.add([x, short_cut])
        x = self.relu(x)
        
        return x

# ResNet Class

In [5]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(64, kernel_size = (7, 7), strides = (1, 1), padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv2 = Conv_Block([64, 64, 256], strides = (1, 1))
        self.conv3 = Conv_Block([128, 128, 512])
        self.conv4 = Conv_Block([256, 256, 1024])
        self.conv5 = Conv_Block([512, 512, 2048])
        
        self.identity_1 = Identity_Block([64, 64, 256])
        self.identity_2 = Identity_Block([128, 128, 512])
        self.identity_3 = Identity_Block([256, 256, 1024])
        self.identity_4 = Identity_Block([512, 512, 2048])
        
        self.global_pool = tf.keras.layers.GlobalMaxPool2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
        
        
        
        
    def call(self, inputs):
        # 1차 Conv
        x = self.conv1(inputs)
        x = self.bn(x)
        x = self.relu(x)
        x = self.max_pool(x)
        
        # Block 1
        x = self.conv2(x)
        x = self.identity_1(x)
        x = self.identity_1(x)
        
        # Block 2
        x = self.conv3(x)
        x = self.identity_2(x)
        x = self.identity_2(x)
        x = self.identity_2(x)
        
        # Block 3
        x = self.conv4(x)
        x = self.identity_3(x)
        x = self.identity_3(x)
        x = self.identity_3(x)
        x = self.identity_3(x)
        x = self.identity_3(x)
        
        # Block 4
        x = self.conv5(x)
        x = self.identity_4(x)
        x = self.identity_4(x)
        
        x = self.global_pool(x)
        
        return self.classifier(x)